In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
# スクレイピングしたいURLの初期ページ
base_url = "https://suumo.jp/jj/bukken/ichiran/JJ010FJ001/?ar=030&bs=011&ta=13&jspIdFlg=patternShikugun&sc=13121&kb=1&kt=9999999&mb=0&mt=9999999&ekTjCd=&ekTjNm=&tj=0&cnb=0&cn=9999999&srch_navi=1"
html = requests.get(base_url)
soup = BeautifulSoup(html.content, "html.parser")

# ページ数を取得するためのページネーション部分の要素を見つける
pagination = soup.find("div", class_="pagination pagination_set-nav")

# 最終ページ番号を含むa要素を探す
last_page_link = pagination.find_all("a")[-2]
# テキストとして最終ページ番号を取得
last_page_number = int(last_page_link.text)
print(last_page_number)
data = []

for page_number in range(1, last_page_number):
    # 各ページのURLを作成
    url = base_url + f"&page={page_number}"

    html = requests.get(url)
    soup = BeautifulSoup(html.content, "html.parser")
    links = soup.select("h2 a")

    prefix_url = "https://suumo.jp"

    for link in links:
        full_url = prefix_url + link.attrs['href']
        more_html = requests.get(full_url)
        more_soup = BeautifulSoup(more_html.content, "html.parser")

        # 特定のclassや要素を指定してテーブルの範囲を抜き出す
        table_name = more_soup.find_all('td', class_='w752 bdCell b')
        name = table_name[0].text.strip()
    
        table_content = more_soup.find_all('td', class_='w290 bdCell')
        # 不要なテキストを含むパターン
        unwanted_patterns = re.compile(r'\[.*\]')
        price = unwanted_patterns.sub('', table_content[0].text).strip()
        layout = unwanted_patterns.sub('', table_content[1].text).strip()
        area = unwanted_patterns.sub('', table_content[4].text).strip()
        floor_info = unwanted_patterns.sub('', table_content[6].text).strip()
        completion_date = unwanted_patterns.sub('', table_content[7].text).strip()
        address = unwanted_patterns.sub('', table_content[8].text).strip()
    
        # 取得したデータを辞書に追加する
        data.append({
            'Name': name,
            'Price': price,
            'Layout': layout,
            'Area': area,
            'FloorInfo': floor_info,
            'CompletionDate': completion_date,
            'Address': address
        })

# リストをデータフレームに変換する
df = pd.DataFrame(data)

34


In [2]:
df

,Name,Price,Layout,Area,FloorInfo,CompletionDate,Address
0,五反野第２スカイハイツ,1630万円,1LDK,47.54m2（14.38坪）（壁芯）,2階/RC6階建,1978年4月,東京都足立区青井２-2-20
1,ローズハイツ千住桜木,1850万円,2LDK,42.03m2（壁芯）,7階/SRC8階建,1979年10月,東京都足立区千住桜木２
2,ボナハイツ中川３号棟,1950万円,3LDK,68.04m2（20.58坪）（壁芯）,2階/SRC14階建,1977年9月,東京都足立区大谷田２-3-35
3,SSKグリーンパーク北綾瀬,1998万円,3LDK,60.5m2（18.30坪）（壁芯）,2階/RC8階建,1989年7月,東京都足立区辰沼２
4,◆竹の塚ハイツ 4階◆,2080万円,3LDK,59.75m2（壁芯）,4階/RC7階建,1984年12月,東京都足立区伊興１
...,...,...,...,...,...,...,...
985,千住ザ・タワー,8980万円,2LDK,66.98m2（壁芯）,16階/RC30階地下1階建,2021年1月,東京都足立区千住１
986,アクアテラブロードスクエア,8980万円,4LDK,115.86m2（35.04坪）（壁芯）,4階/RC14階地下1階建,2008年1月,東京都足立区新田３
987,千住ザ・タワー,9180万円,1LDK+S（納戸）,66.98m2（20.26坪）（壁芯）,15階/RC30階地下1階建,2021年1月,東京都足立区千住１
988,千住ザ・タワー,9900万円,3LDK,71.54m2（21.64坪）（壁芯）,25階/RC30階地下1階建,2021年1月,東京都足立区千住１


In [3]:
# name, price, floorinfoの３つが同一である行を重複とみなして削除する
df.drop_duplicates(subset=['Name', 'Price', 'FloorInfo'], keep='first', inplace=True)

In [4]:
df

,Name,Price,Layout,Area,FloorInfo,CompletionDate,Address
0,五反野第２スカイハイツ,1630万円,1LDK,47.54m2（14.38坪）（壁芯）,2階/RC6階建,1978年4月,東京都足立区青井２-2-20
1,ローズハイツ千住桜木,1850万円,2LDK,42.03m2（壁芯）,7階/SRC8階建,1979年10月,東京都足立区千住桜木２
2,ボナハイツ中川３号棟,1950万円,3LDK,68.04m2（20.58坪）（壁芯）,2階/SRC14階建,1977年9月,東京都足立区大谷田２-3-35
3,SSKグリーンパーク北綾瀬,1998万円,3LDK,60.5m2（18.30坪）（壁芯）,2階/RC8階建,1989年7月,東京都足立区辰沼２
4,◆竹の塚ハイツ 4階◆,2080万円,3LDK,59.75m2（壁芯）,4階/RC7階建,1984年12月,東京都足立区伊興１
...,...,...,...,...,...,...,...
985,千住ザ・タワー,8980万円,2LDK,66.98m2（壁芯）,16階/RC30階地下1階建,2021年1月,東京都足立区千住１
986,アクアテラブロードスクエア,8980万円,4LDK,115.86m2（35.04坪）（壁芯）,4階/RC14階地下1階建,2008年1月,東京都足立区新田３
987,千住ザ・タワー,9180万円,1LDK+S（納戸）,66.98m2（20.26坪）（壁芯）,15階/RC30階地下1階建,2021年1月,東京都足立区千住１
988,千住ザ・タワー,9900万円,3LDK,71.54m2（21.64坪）（壁芯）,25階/RC30階地下1階建,2021年1月,東京都足立区千住１


In [5]:
# スプレッドシート形式で出力する
df.to_excel('output.xlsx', index=False)  # index=Falseでインデックスを出力しないようにしています